# Timer

In [1]:
from datetime import datetime as dt

start_time = "25.11.2022, 11:15"
start_time = dt.strptime(start_time, "%d.%m.%Y, %H:%M")

while True:
    if start_time < dt.now():
        break
    else:
        sleep(10)

# Parameter optimization

In [1]:
import sys
sys.path.append('..')

import numpy as np
import pandas as pd
from glob import glob
from optimizer import Optimizer
from os import environ

# Set environment variable
environ["ENV"] = "optimize"

from config.config import ConfigFactory

pd.set_option('display.max_columns', 500)

ttype = 'buy'
pattern = ['STOCH', 'RSI', 'LinearReg']
indicator_list = pattern
indicator_list_higher = ['LinearReg']

work_timeframe = '15m'
higher_timeframe = '1h'
opt_limit = 1000
load = False

print(f'Timeframe is {work_timeframe}/{higher_timeframe}, trade type is {ttype}')

# Get configs
configs = ConfigFactory.factory(environ).configs
configs['Indicator_list'] = indicator_list
configs['Higher_TF_indicator_list'] = indicator_list_higher
configs['Timeframes']['work_timeframe'] = work_timeframe
configs['Timeframes']['higher_timeframe'] = higher_timeframe

optim_dict = {
                'RSI': {'timeperiod': [14], 'low_bound': [35]},
                'STOCH': {'fastk_period': [9], 'slowk_period': [7],
                          'slowd_period': [3], 'low_bound': [25]},
                'LinearReg': {'timeperiod': np.arange(2, 102, 2), 'low_bound': [0]}
             }

opt = Optimizer(pattern, optim_dict, **configs)
stat = opt.optimize(pattern, ttype, opt_limit, load)

stat_list = glob(f'opt_{"_".join(pattern)}_{ttype}_{work_timeframe}_{higher_timeframe}*')
if not stat_list:
    stat.to_pickle(f'opt_{"_".join(pattern)}_{ttype}_{work_timeframe}_{higher_timeframe}.pkl')
else:
    stat.to_pickle(f'opt_{"_".join(pattern)}_{ttype}_{work_timeframe}_{higher_timeframe}_{len(stat_list)}.pkl')


Timeframe is 15m/1h, trade type is buy
Number of combinations is 50


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 50/50 [14:15<00:00, 17.12s/it]


# Check local statistics

In [11]:
# candles
e_ratio_cols = [f'e_ratio_{lag + 1}' for lag in range(24)]
pct_price_diff_cols = [f'pct_price_diff_{lag + 1}' for lag in range(24)]
pct_price_diff_cols = [f'pct_price_diff_{lag + 1}' for lag in range(24)]

stat['e_ratio_avg'] = stat[[f'e_ratio_{lag + 1}' for lag in range(24)]].apply(np.mean, axis=1)
stat['mar_avg'] = stat[[f'pct_price_diff_{lag + 1}' for lag in range(24)]].apply(np.mean, axis=1)
stat['e_ratio_rank'] = (stat['e_ratio_avg'] - 1) * stat['forecasts_num']
stat['mar_rank'] = stat['mar_avg'] * stat['forecasts_num']

stat = stat[[c for c in stat.columns if c not in e_ratio_cols + pct_price_diff_cols] + e_ratio_cols + 
            pct_price_diff_cols].sort_values('mar_avg', ascending=False)
stat.head(20)

,pattern,RSI_timeperiod,RSI_low_bound,STOCH_fastk_period,STOCH_slowk_period,STOCH_slowd_period,STOCH_low_bound,LinearReg_timeperiod,LinearReg_low_bound,forecasts_num,e_ratio_avg,mar_avg,e_ratio_rank,mar_rank,e_ratio_1,e_ratio_2,e_ratio_3,e_ratio_4,e_ratio_5,e_ratio_6,e_ratio_7,e_ratio_8,e_ratio_9,e_ratio_10,e_ratio_11,e_ratio_12,e_ratio_13,e_ratio_14,e_ratio_15,e_ratio_16,e_ratio_17,e_ratio_18,e_ratio_19,e_ratio_20,e_ratio_21,e_ratio_22,e_ratio_23,e_ratio_24,pct_price_diff_1,pct_price_diff_2,pct_price_diff_3,pct_price_diff_4,pct_price_diff_5,pct_price_diff_6,pct_price_diff_7,pct_price_diff_8,pct_price_diff_9,pct_price_diff_10,pct_price_diff_11,pct_price_diff_12,pct_price_diff_13,pct_price_diff_14,pct_price_diff_15,pct_price_diff_16,pct_price_diff_17,pct_price_diff_18,pct_price_diff_19,pct_price_diff_20,pct_price_diff_21,pct_price_diff_22,pct_price_diff_23,pct_price_diff_24
6,STOCH_RSI_LinearReg,14,35,9,7,3,25,14,0,94,1.580921,6.205417,54.606558,583.309167,1.3110,1.2998,1.5298,1.6477,1.8456,1.8092,1.7642,1.7300,1.6757,1.6974,1.6906,1.7729,1.7379,1.7326,1.6465,1.5833,1.4295,1.4483,1.4423,1.4405,1.4430,1.3980,1.4319,1.4344,2.21,3.5,4.00,5.92,5.29,8.98,9.75,10.00,10.00,10.00,10.00,10.00,6.67,6.77,4.23,3.85,4.23,4.60,4.95,4.56,4.39,4.88,5.19,4.96
7,STOCH_RSI_LinearReg,14,35,9,7,3,25,16,0,117,1.461254,5.852083,53.966737,684.693750,1.2320,1.2101,1.3329,1.3429,1.5094,1.4583,1.4749,1.4560,1.4231,1.4872,1.5483,1.5898,1.5572,1.5766,1.5735,1.5459,1.4327,1.4510,1.4442,1.4625,1.4788,1.4718,1.5054,1.5056,2.21,3.5,4.00,5.92,5.29,8.98,9.75,10.00,10.00,10.00,10.00,10.00,6.67,5.10,3.68,3.53,3.12,3.50,3.87,3.80,3.84,4.31,4.73,4.65
8,STOCH_RSI_LinearReg,14,35,9,7,3,25,18,0,138,1.428092,5.658750,59.076650,780.907500,1.2778,1.2035,1.2972,1.2688,1.4182,1.3521,1.3837,1.3469,1.3380,1.4180,1.4757,1.5304,1.5187,1.5467,1.5494,1.5562,1.4655,1.4845,1.4696,1.4895,1.4861,1.4545,1.4730,1.4702,1.12,3.5,4.00,5.92,5.29,8.98,9.75,10.00,10.00,10.00,10.00,10.00,6.67,5.10,3.16,3.11,2.84,3.20,3.55,3.54,3.61,4.05,4.21,4.21
3,STOCH_RSI_LinearReg,14,35,9,7,3,25,8,0,26,3.290996,5.402083,59.565892,140.454167,2.6930,3.3175,4.1100,4.3732,4.1709,3.9749,3.6125,3.6038,3.5464,3.5648,3.5784,3.6707,3.6121,3.8224,3.6040,3.2799,2.7743,2.6842,2.5855,2.5979,2.4802,2.4246,2.4390,2.4637,2.21,3.5,4.00,5.92,5.29,8.98,9.75,10.00,6.69,6.72,6.75,6.78,5.13,5.24,4.33,3.76,3.94,4.15,4.34,3.95,3.73,4.13,5.07,5.29
4,STOCH_RSI_LinearReg,14,35,9,7,3,25,10,0,48,1.747742,5.380417,35.891600,258.260000,1.8744,1.7282,2.0343,2.1609,2.0678,2.0026,1.8836,1.8772,1.8492,1.8333,1.8268,1.8537,1.8059,1.8844,1.7799,1.7172,1.5017,1.4681,1.4498,1.4948,1.4496,1.4408,1.4740,1.4876,2.21,3.5,4.00,5.92,5.29,8.98,9.75,10.00,6.69,6.72,6.75,6.78,5.13,5.24,4.33,3.76,3.94,4.15,4.34,3.95,3.73,4.13,5.07,4.77
5,STOCH_RSI_LinearReg,14,35,9,7,3,25,12,0,75,1.566725,5.380000,42.504375,403.500000,1.5853,1.3381,1.6355,1.7697,1.9865,1.9101,1.8387,1.8244,1.7415,1.6933,1.6745,1.6413,1.5862,1.5802,1.5290,1.5014,1.3400,1.3397,1.3132,1.3266,1.3386,1.3359,1.3815,1.3902,2.21,3.5,4.00,5.92,5.29,8.98,9.75,10.00,6.69,6.72,6.75,6.78,5.13,5.24,4.33,3.76,3.94,4.15,4.34,3.95,3.73,4.13,5.07,4.76
11,STOCH_RSI_LinearReg,14,35,9,7,3,25,24,0,212,1.331921,5.072917,70.367217,1075.458333,1.1587,1.0771,1.1211,1.1650,1.2656,1.2126,1.2309,1.2238,1.2538,1.3077,1.3517,1.4174,1.4250,1.4614,1.4809,1.4897,1.4313,1.4421,1.4177,1.4299,1.4222,1.4096,1.4091,1.3618,1.12,3.5,4.00,2.97,2.91,4.97,4.40,4.71,8.67,8.89,7.33,7.67,6.73,5.46,4.72,5.41,4.98,4.96,5.26,4.68,4.49,4.77,4.57,4.58
10,STOCH_RSI_LinearReg,14,35,9,7,3,25,22,0,184,1.439013,5.038333,80.778300,927.053333,1.1903,1.1632,1.2334,1.2722,1.3956,1.3429,1.3488,1.3400,1.3540,1.4163,1.4721,1.5383,1.5581,1.5833,1.6121,1.6227,1.5423,1.5548,1.5151,1.5256,1.5097,1.4834,1.4871,1.4750,1.12,3.5,4.00,2.97,2.91,4.97,4.40,4.71,8.67,8.89,7.33,7.67,6.73,6.13,4.65,5.02,4.62,4.63,4.94,4.38,4.43,4.73,4.76,4.76
13,STOCH_RSI_LinearReg,14,35,9,7,3,25,28,0,248,1.356683,4.996667,88.457467,1239.173333,1.1409,1.0765,1.1220,1.1526,1.2325,

# Research parameter influence

In [16]:
param = 'LinearReg_timeperiod'

res = stat.groupby([param]).agg({'e_ratio_avg': 'mean',
                                 'pct_price_diff_avg': 'mean',
                                 'e_ratio_rank': 'mean', 
                                 'price_rank': 'mean', 
                                 'forecasts_num': 'sum'}).sort_values(param).reset_index()
res

,LinearReg_timeperiod,e_ratio_avg,pct_price_diff_avg,e_ratio_rank,price_rank,forecasts_num
0,2,3.236211,1.144583,86.068997,44.164444,114
1,3,5.849584,1.552812,62.963132,19.165312,56
2,4,6.061108,1.460000,53.063822,15.588472,32
3,5,6.842036,1.795556,47.689653,14.873611,25
4,6,5.730101,1.770000,37.715247,13.606111,26
5,8,5.441139,1.488750,57.024707,21.980104,61


# Plot result in file

In [11]:
import matplotlib.pyplot as plt

fig, ax = plt.subplots()

x, y = 'low_bound', 'pct_price_diff_avg'
ax.plot(res[x], res[y])
ax.set_xlabel(x)
ax.set_ylabel(y)
ax.set_title('_'.join(pattern) + '_' + ttype)

fig.savefig('temp.png', dpi=fig.dpi)

KeyError: 'low_bound'

# Save new config data to config file

In [25]:
from config_updater import ConfigUpdater

ttype = 'buy'
pattern = ['STOCH', 'RSI']
work_timeframe = '15m'
higher_timeframe = '1h'
timeframe = f'{work_timeframe}_{higher_timeframe}'
        
optim_dict = {
                'RSI': {'timeperiod': [14], 'low_bound': [35]},
                'STOCH': {'fastk_period': [9], 'slowk_period': [7],
                          'slowd_period': [3], 'low_bound': [25]},
                'LinearReg': {'timeperiod': [3], 'low_bound': [0]}
              }
        
cu = ConfigUpdater(ttype, timeframe)
cu.config_update(optim_dict)

# Check global statistics

In [2]:
import numpy as np
import pandas as pd
from glob import glob

ttype = 'buy'
pattern = ['STOCH', 'RSI']
work_timeframe = '5m'
higher_timeframe = '1h'
stat_list = glob(f'opt_{"_".join(pattern)}_{ttype}_{work_timeframe}_{higher_timeframe}*')
stat = None

for sl in stat_list:
    tmp = pd.read_pickle(sl)
    if stat is None:
        stat = tmp.copy()
    else:
        stat = pd.concat([stat, tmp])
        
stat['pct_right_forecast_avg'] = stat[[f'pct_right_forecast_{lag + 1}' for lag in range(24)]].apply(np.mean, axis=1)
stat['pct_price_diff_avg'] = stat[[f'pct_price_diff_{lag + 1}' for lag in range(24)]].apply(np.mean, axis=1)
stat['forecast_rank'] = (stat['pct_right_forecast_avg'] - 70) * stat['forecasts_num']
stat['price_rank'] = stat['pct_price_diff_avg'] * stat['forecasts_num']
        
total_stat = stat.groupby(['RSI_timeperiod', 
                           'RSI_low_bound', 
                           'STOCH_fastk_period', 
                           'STOCH_slowk_period', 
                           'STOCH_slowd_period', 
                           'STOCH_low_bound']).agg({'pct_right_forecast_avg': 'mean',
                                                    'pct_price_diff_avg': 'mean',
                                                    'forecast_rank': 'mean', 
                                                    'price_rank': 'mean', 
                                                    'forecasts_num': 'sum'}).sort_values('forecast_rank', 
                                                                                         ascending=False)
total_stat.head(20)

pct_right_forecast_avg  \
RSI_timeperiod RSI_low_bound STOCH_fastk_period STOCH_slowk_period STOCH_slowd_period STOCH_low_bound                           
16             30            5                  4                  3                  10                            75.759735   
                                                                   5                  10                            83.794091   
               35            11                 3                  9                  20                            66.518698   
               25            7                  3                  5                  10                            88.219697   
12             20            7                  3                  5                  10                            86.237538   
               30            11                 3                  9                  25                            67.022344   
               25            5                  4                  5                  10                            79.443447   
14             30            5                  4                  5                  10                            72.603182   
12             20            7                  4                  5                  10                            59.343523   
16             25            9                  3                  5                  10                            73.011894   
                             7                  3                  7                  10                            56.439394   
12             20            7                  3                  7                  10                            65.530303   
16             25            7                  4                  5                  10                            54.286970   
                             5                  3                  7                  10                            58.333333   
12             20            5                  3                  7                  10                            58.333333   
                             9                  3                  5                  10                            73.201098   
               30            5                  4                  5                  10                            69.466932   
16             25            7                  2                  5                  10                            77.102348   
               35            5                  4                  5                  10                            69.892652   
14             20            5                  5                  5                  10                            75.757576   

                                                                                                       pct_price_diff_avg  \
RSI_timeperiod RSI_low_bound STOCH_fastk_period STOCH_slowk_period STOCH_slowd_period STOCH_low_bound                       
16             30            5                  4                  3                  10                         0.843144   
                                                                   5                  10                         1.436705   
               35            11                 3                  9                  20                         0.455156   
               25            7                  3                  5                  10                         1.885038   
12             20            7                  3                  5                  10                         1.803295   
               30            11                 3                  9                  25                         0.418698   
               25            5                  4                  5                  10                         1.366742   
14             30            5                  4                  5                  10                         0.644697   
12            